## Build A Motion Detected Alarm System with Python and OpenCV

### import

In [1]:
import cv2
import math
import time

DEBUG_MODE = False

### get euclidean distance between bounding boxes

distance between bounding boxes

In [2]:
def get_euclidean_distance(previous_box, current_box):
    x_len = (previous_box['x0'] + previous_box['x1'] / 2) - (current_box['x0'] + current_box['x1'] / 2)
    y_len = (previous_box['y0'] + previous_box['y1'] / 2) - (current_box['y0'] + current_box['y1'] / 2)
    return math.sqrt(x_len ** 2 + y_len ** 2) 

def get_difference_of_coordinates(current_box, previous_box):
    return [current_box['x0'] - previous_box['x0'], current_box['y0'] - previous_box['y0']]    

### are similar area

In [3]:
def are_similar_area(previous_box, min_euclidean_index, bounding_boxes):
    if 0 > min_euclidean_index:
        return False
    previous_area = abs((previous_box['x0'] - previous_box['x1']) * (previous_box['y0'] - previous_box['y1']))        
    current_box = bounding_boxes[min_euclidean_index]
    current_area = abs((current_box['x0'] - current_box['x1']) * (current_box['y0'] - current_box['y1']))    

    if previous_area > (current_area * 1.5):
        return False
    elif ( previous_area * 1.5 ) < current_area:
        return False
    else:       
        return True

### is car

In [4]:
def is_Car(current_box, X_CROSSWALK, X_DIFF_INITIAL, Y_DIFF_INITIAL):
    if current_box['counted_as_a_car_to_city_centre']:
        return True
    if current_box['x_diff'] == X_DIFF_INITIAL:
        return False
    if (current_box['x_diff'] == 0 and current_box['y_diff'] == 0 and 
        X_CROSSWALK < current_box['x0'] < X_CROSSWALK + 60):
        return False
    if is_Pedestrian(current_box, X_CROSSWALK):
        return False
    return True

### is pedestrian

In [5]:
def is_Pedestrian(current_box, X_CROSSWALK):
    if (not current_box['counted_as_a_pedestrian'] and
        X_CROSSWALK < current_box['x0'] < X_CROSSWALK + 60): 
        if ( current_box['x_diff'] == 0 and abs(current_box['y_diff']) <= 2 ):
            current_box['counted_as_a_pedestrian'] = True
        elif ( abs(current_box['x_diff']) == 1 and current_box['y_diff'] == 0 and
            cv2.contourArea(current_box['contour']) < 1000 * 4
        ):
            current_box['counted_as_a_pedestrian'] = True
#        elif ( abs(current_box['x_diff']) < 2 and
#            abs(current_box['y_diff'] > 20) and
#            cv2.contourArea(c) < 1000 * 2.9             
#        ):
#            current_box['is_counted_as_a_pedestrian'] = True
    return current_box['counted_as_a_pedestrian']



### check if a car to city centre

In [6]:
def check_if_a_car_to_city_centre(current_box, X_CHECK, Y_CHECK):
    current_id = current_box['id']
    current_x0 = current_box['x0']
    current_y0 = current_box['y0']
    current_x1 = current_box['x1']
    current_y1 = current_box['y1']
    x_diff = current_box['x_diff']
    y_diff = current_box['y_diff']
    
    if (not current_box['counted_as_a_car_to_city_centre'] and 
        current_x0 < X_CHECK < current_x1 and current_y1 < Y_CHECK and 
        x_diff < 0        
       ):
        current_box['counted_as_a_car_to_city_centre'] = True
        return True
    return False

### draw bounding box

In [7]:
def draw_Bounding_Box_and_Text(current_box, color_rect, frame, DEBUG_MODE): 
    current_id = current_box['id']
    current_x0 = current_box['x0']
    current_y0 = current_box['y0']
    current_x1 = current_box['x1']
    current_y1 = current_box['y1']

    if DEBUG_MODE:
        cv2.putText(frame, f'{current_id}', (current_x0, current_y0), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)     
    cv2.rectangle(frame, (current_x0, current_y0), (current_x1, current_y1), color_rect, 1)
    


In [8]:
### print bonding box information

In [9]:
def print_Bounding_Box_Info(current_box):
    id = current_box['id']
    x0 = current_box['x0']
    y0 = current_box['y0']
    x1 = current_box['x1']
    y1 = current_box['y1']
    x_diff = current_box['x_diff']
    y_diff = current_box['y_diff']
    counted_as_pestrian = current_box['counted_as_a_pedestrian']
    print(f'{id} / {frame_number} / {(x0+x1)/2}, {(y0+y1)/2} / {x_diff} , {y_diff} / {x0},{y0},{x1},{y1} / {counted_as_pestrian}')

### detect cars from video

In [10]:
def detect_Cars_from_Video(video_name):
    # Create a VideoCapture object and read the frames from an input file
    video=cv2.VideoCapture(video_name)
 
    TOTAL_WIDTH = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    TOTAL_HEIGHT = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    MAX_DISTANCE = math.sqrt(TOTAL_WIDTH * TOTAL_WIDTH + TOTAL_HEIGHT * TOTAL_HEIGHT)

    X_DOWNTOWN = TOTAL_WIDTH // 2 + 143    
    Y_MAIN_STREET = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT) / 2) + 40
    
    MIN_CAR_AREA = 1000 * 2.8
    MAX_CAR_AREA = 1000 * 30

    X_CROSSWALK = 540
    X_DIFF_INITIAL = 100000
    Y_DIFF_INITIAL = 100000
    
    # Check if the video opened successfully
    if (video.isOpened()== False): 
      print("Error opening video file")
    
    initial_frame = None
    box_id = -1
    bounding_boxes = [ [] for _ in range(int(video.get(cv2.CAP_PROP_FRAME_COUNT))) ] 
    number_of_cars_to_city_centre = 0
    
    # # Read until video is completed or we press 'q'
    for frame_number in range(int(video.get(cv2.CAP_PROP_FRAME_COUNT))):
        # Capture frame-by-frame
        # Note that VideoCapture captures the frames of a video without considering the fps of the video
        check, frame = video.read()
        status = 0
        
        if not check:
            break
            
        # Gray conversion and noise reduction (smoothening)
        gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)    
        blur_frame=cv2.GaussianBlur(gray_frame,(13, 13),0)
    
        if initial_frame is None:
            initial_frame = blur_frame
            continue
    
        delta_frame=cv2.absdiff(initial_frame,blur_frame)
        
        threshold_frame=cv2.threshold(delta_frame, 5, 255, cv2.THRESH_BINARY)[1]    
        (contours,_)=cv2.findContours(threshold_frame,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)    
    
        for c in contours:
            # contourArea() method filters out any small contours
            # You can change this value
            if not (MIN_CAR_AREA < cv2.contourArea(c) < MAX_CAR_AREA):            
                continue
           
            status=status + 1
            (x, y, w, h)=cv2.boundingRect(c)
    
            if Y_MAIN_STREET > y :
                continue
                    
            box_id = box_id + 1       
            bounding_boxes[frame_number].append({
                'id': box_id, 'x0': x, 'y0': y, 'x1': x+w, 'y1': y+h, 
                'x_diff': X_DIFF_INITIAL, 'y_diff': Y_DIFF_INITIAL,                            
                'counted_as_a_car_to_city_centre': False, 'counted_as_a_pedestrian': False,
                'contour': c
            })
        cv2.rectangle(frame, (0, Y_MAIN_STREET), (TOTAL_WIDTH, TOTAL_HEIGHT), (0, 0, 255) , 1)
        cv2.line(frame, (X_DOWNTOWN, Y_MAIN_STREET), (X_DOWNTOWN, TOTAL_HEIGHT- 120), (255, 0, 0), 1)
        
        if frame_number >= 1:
            for previous_box in bounding_boxes[frame_number -1]:
                min_euclidean_distance = MAX_DISTANCE
                min_euclidean_index = -1
                for i, current_box in enumerate(bounding_boxes[frame_number]):
                    current_euclidean_distance = get_euclidean_distance(previous_box, current_box)
                    if current_euclidean_distance <= min_euclidean_distance:
                        min_euclidean_distance = current_euclidean_distance
                        min_euclidean_index = i
                if are_similar_area(previous_box, min_euclidean_index, bounding_boxes[frame_number]):                
                    bounding_boxes[frame_number][min_euclidean_index]['id'] = previous_box['id']
                    bounding_boxes[frame_number][min_euclidean_index]['counted_as_a_car_to_city_centre'] = previous_box['counted_as_a_car_to_city_centre'] 
                    bounding_boxes[frame_number][min_euclidean_index]['counted_as_a_pedestrian'] = previous_box['counted_as_a_pedestrian']                 
                    bounding_boxes[frame_number][min_euclidean_index]['x_diff'],  bounding_boxes[frame_number][min_euclidean_index]['y_diff'] = get_difference_of_coordinates(bounding_boxes[frame_number][min_euclidean_index], previous_box)

        number_of_cars_increased = False
        for current_box in bounding_boxes[frame_number]:
            if is_Car(current_box, X_CROSSWALK, X_DIFF_INITIAL, Y_DIFF_INITIAL):
                color_rect = (0, 255, 0)
                if check_if_a_car_to_city_centre(current_box, X_DOWNTOWN, TOTAL_HEIGHT * 7 // 8):
                    number_of_cars_increased = True
                    number_of_cars_to_city_centre = number_of_cars_to_city_centre + 1
                    color_rect = (255, 0, 0)
                draw_Bounding_Box_and_Text(current_box, color_rect, frame, DEBUG_MODE)
            if DEBUG_MODE and current_box['x0'] > 500:
                print_Bounding_Box_Info(current_box)

        if DEBUG_MODE:
            cv2.putText(frame, f'{ len(bounding_boxes[frame_number]) }', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2) 
        cv2.putText(frame, f'cars to the city centre: { number_of_cars_to_city_centre }', (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2) 
        
        if check == True:
            cv2.imshow("movie",frame)     
            if DEBUG_MODE:
                cv2.imshow('Threshold frame', threshold_frame)        
            
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
                
            if number_of_cars_increased:
                time.sleep(2)
        else:
            break
    
    # After the loop release the video object
    video.release()
    
    # Destroy all the windows
    cv2.destroyAllWindows()
    return number_of_cars_to_city_centre

In [11]:
### run 

In [12]:
print(detect_Cars_from_Video('Traffic_Laramie_1.mp4'))
print(detect_Cars_from_Video('Traffic_Laramie_2.mp4'))

6
4
